### TODO:

* Different font size baselines don't line-up within a line.
* Need to sync PDF rendering with new HTML markup.

In [1]:
from PIL import Image, ImageDraw

import toyplot.font
import toyplot.text

import IPython.display

import xml.etree.ElementTree as xml
import sys
import logging
logging.basicConfig(level=logging.DEBUG)
toyplot.log.setLevel(logging.DEBUG)

In [2]:
#text = """This is <i>italic</i>, <b>bold</b>,<br/><code>monospace</code>, <small>small</small>, <span style="font-size:24px">large</span>,<br/><sup>superscript</sup> and <sub>subscript</sub> text<br/>for your <span style="fill:red">amusement!</span>"""
#text = """<span style="alignment-baseline:hanging">Hanging</span><span style="alignment-baseline:central">Central</span><span style="alignment-baseline:middle">Middle</span><span style="alignment-baseline:alphabetic">Alpha</span>"""
text = """Hello, <small>World!</small>"""
style = {"font-size":"24px", "-toyplot-text-baseline":"visible"}
fonts = toyplot.font.ReportlabLibrary()

layout = toyplot.text.layout(text=text, style=style, fonts=fonts)

In [3]:
toyplot.text.dump(layout)

toyplot.text.Layout
  bottom: 14.2992
  height: 28.8
  left: -61.7424
  right: 61.7424
  top: -14.5008
  width: 123.4848

  toyplot.text.LineBox
    baseline: 0.0
    bottom: 14.2992
    height: 28.8
    left: -61.7424
    right: 61.7424
    top: -14.5008
    width: 123.4848

    toyplot.text.TextBox
      baseline: 6.0312
      bottom: 20.3304
      left: -61.7424
      right: 6.2736
      style:
        font-size: 24.0
        -toyplot-text-baseline: visible
        white-space: pre
        vertical-align: baseline
        font-family: helvetica
        line-height: 28.8
      text: 'Hello, '
      top: -8.4696
      width: 68.016

    toyplot.text.TextBox
      baseline: 4.82496
      bottom: 16.26432
      left: 6.2736
      right: 61.7424
      style:
        font-size: 19.2
        -toyplot-text-baseline: visible
        white-space: pre
        vertical-align: baseline
        font-family: helvetica
        line-height: 23.04
      text: 'World!'
      top: -6.77568
      width:

In [4]:
canvas = toyplot.Canvas(width="5in", height="2in")
axes = canvas.cartesian(show=False)
axes.text(0, 0, text=text, style=style)
axes.scatterplot(0, 0, color="black", size=3);

Hello, World! 
 
 
 Save as .csv

In [5]:
import toyplot.pdf
toyplot.pdf.render(canvas, "test.pdf")